### Importing Libraries and IBM PAIRS Authentication

In [7]:
import pandas as pd
from ibmpairs import paw, authentication
import json
import numpy
from datetime import datetime, timedelta
import logging
import matplotlib.pyplot as plt
import re
import numpy as np

In [29]:
logging.basicConfig(level=logging.INFO)
pawLogger = logging.getLogger('ibmpairs.paw')
pawLogger.setLevel(logging.INFO)

In [30]:
with open("ibmpairskey.txt", "r") as f:
    pairs_key = f.read()

In [31]:
pairs_credentials = authentication.OAuth2(api_key=pairs_key)
auth_header = {'Authorization': f'Bearer {pairs_credentials.jwt_token}'}
PAIRS_SERVER   = 'https://pairs.res.ibm.com'
PAIRS_CREDENTIALS = authentication.OAuth2(api_key=pairs_key)

### Loop over dates to retrieve Sentinel-2 data 

In [14]:
rgbNirNdviLayers = {
    'Red' : '49360',
    'Green' : '49681',
    'Blue' : '49680',
    'NIR': '49361',
    'NDVI' : '49464',
    'SCL': '49362'
}

bbox = ["46.94599", "7.68597", "47.20599", "7.94597"] # around Bern

date = None
queryJson = {
    'layers' : [
        {
            
            'type' : 'raster', 'id' : rgbNirNdviLayers[lKey]
            # 'temporal' : {'snapshot' : [{date}]},
            # 'output' : False
        }
        for lKey in rgbNirNdviLayers
    ],
    "spatial" : {"type" : "square",  "coordinates" : bbox }, 
    "temporal" : {"intervals" : [{"snapshot" : date}]} 
}

In [15]:
from datetime import date, timedelta

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2015, 8, 1)
end_date = date(2015, 8, 5)

iso8601 = '%Y-%m-%dT%H:%M:%SZ'

for single_date in daterange(start_date, end_date):
    date = single_date.strftime(iso8601)
    queryJson["temporal"]["intervals"][0]["snapshot"] = date
    
    print(queryJson)
    
    try:    
        query = paw.PAIRSQuery(queryJson, PAIRS_SERVER, PAIRS_CREDENTIALS, authType='api-key', overwriteExisting=False)
        query.submit()

        query.poll_till_finished()
        query.download()
        query.create_layers()
        # Could add some data manipulation here and then save the data
    except:
        print('No data for this date')

{'layers': [{'type': 'raster', 'id': '49360'}, {'type': 'raster', 'id': '49681'}, {'type': 'raster', 'id': '49680'}, {'type': 'raster', 'id': '49361'}, {'type': 'raster', 'id': '49464'}, {'type': 'raster', 'id': '49362'}], 'spatial': {'type': 'square', 'coordinates': ['46.94599', '7.68597', '47.20599', '7.94597']}, 'temporal': {'intervals': [{'snapshot': '2015-08-01T00:00:00Z'}]}}
No data for this date
{'layers': [{'type': 'raster', 'id': '49360'}, {'type': 'raster', 'id': '49681'}, {'type': 'raster', 'id': '49680'}, {'type': 'raster', 'id': '49361'}, {'type': 'raster', 'id': '49464'}, {'type': 'raster', 'id': '49362'}], 'spatial': {'type': 'square', 'coordinates': ['46.94599', '7.68597', '47.20599', '7.94597']}, 'temporal': {'intervals': [{'snapshot': '2015-08-02T00:00:00Z'}]}}
No data for this date
{'layers': [{'type': 'raster', 'id': '49360'}, {'type': 'raster', 'id': '49681'}, {'type': 'raster', 'id': '49680'}, {'type': 'raster', 'id': '49361'}, {'type': 'raster', 'id': '49464'}, {

### Loop over dates to retrieve ERA-5 data 

In [16]:
eraLayers = {
    'Total Prec' : '49459',
    'SP' : '49439',
    'Temp' : '49423',
    'AWVC' : '49458'
}

date_start, date_end = None, None

bbox = ["46.94599", "7.68597", "47.20599", "7.94597"] # around Bern


queryJson = {
    'layers' : [
        {            
            'type' : 'raster', 'id' : eraLayers[lKey],
            "aggregation" : "Mean"
            #"temporal" : {"intervals" : [{"start" : date_start, "end" : date_end}]} # if you put date here, need to update them in loop (for every layer)
        }
        for lKey in eraLayers
    ],
    "spatial" : {"type" : "square",  "coordinates" : bbox }, 
    "temporal" : {"intervals" : [
      {"start" : date_start, "end" : date_end}
  ]}
}

In [17]:
from datetime import date, datetime, timedelta

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = datetime(2016, 6, 1, 12)
end_date = datetime(2016, 6, 14, 12)
iso8601 = '%Y-%m-%dT%H:%M:%SZ'

for single_date in daterange(start_date, end_date):
    date_start = single_date.strftime(iso8601)
    date_end = (single_date + timedelta(1)).strftime(iso8601)
    queryJson["temporal"]["intervals"][0]["start"] = date_start
    queryJson["temporal"]["intervals"][0]["end"] = date_end    
    print(queryJson)
    
    try:    
        query = paw.PAIRSQuery(queryJson, PAIRS_SERVER, PAIRS_CREDENTIALS, authType='api-key', overwriteExisting=False)
        query.submit()

        query.poll_till_finished()
        query.download()
        query.create_layers()
        # Could add some data manipulation here and then save the data
    except:
        print('No data for this date')

{'layers': [{'type': 'raster', 'id': '49459', 'aggregation': 'Mean'}, {'type': 'raster', 'id': '49439', 'aggregation': 'Mean'}, {'type': 'raster', 'id': '49423', 'aggregation': 'Mean'}, {'type': 'raster', 'id': '49458', 'aggregation': 'Mean'}], 'spatial': {'type': 'square', 'coordinates': ['46.94599', '7.68597', '47.20599', '7.94597']}, 'temporal': {'intervals': [{'start': '2016-06-01T12:00:00Z', 'end': '2016-06-02T12:00:00Z'}]}}
No data for this date
{'layers': [{'type': 'raster', 'id': '49459', 'aggregation': 'Mean'}, {'type': 'raster', 'id': '49439', 'aggregation': 'Mean'}, {'type': 'raster', 'id': '49423', 'aggregation': 'Mean'}, {'type': 'raster', 'id': '49458', 'aggregation': 'Mean'}], 'spatial': {'type': 'square', 'coordinates': ['46.94599', '7.68597', '47.20599', '7.94597']}, 'temporal': {'intervals': [{'start': '2016-06-02T12:00:00Z', 'end': '2016-06-03T12:00:00Z'}]}}
No data for this date
{'layers': [{'type': 'raster', 'id': '49459', 'aggregation': 'Mean'}, {'type': 'raster',

### Only query ERA when Sentinel 2 is available

In [20]:
#Layers,Dates and BBOX

date_start, date_end = None, None
bbox = ["46.94599", "7.68597", "47.20599", "7.94597"] # around Bern

rgbNirNdviLayers = {
    'Red' : '49360',
    'Green' : '49681',
    'Blue' : '49680',
    'NIR': '49361',
    'NDVI' : '49464',
    'SCL': '49362'
}

eraLayers = {
    'Total Prec' : '49459',
    'SP' : '49439',
    'Temp' : '49423',
    'AWVC' : '49458'
}

In [21]:
queryEraJson = {
    'layers' : [
        {            
            'type' : 'raster', 'id' : eraLayers[lKey],
            "aggregation" : "Mean"
            #"temporal" : {"intervals" : [{"start" : date_start, "end" : date_end}]} # if you put date here, need to update them in loop (for every layer)
        }
        for lKey in eraLayers
    ],
    "spatial" : {"type" : "square",  "coordinates" : bbox }, 
    "temporal" : {"intervals" : [
      {"start" : date_start, "end" : date_end}
  ]}
}

In [22]:
querySentinelJson = {
    'layers' : [
        {
            
            'type' : 'raster', 'id' : rgbNirNdviLayers[lKey]
            # 'temporal' : {'snapshot' : [{date}]},
            # 'output' : False
        }
        for lKey in rgbNirNdviLayers
    ],
    "spatial" : {"type" : "square",  "coordinates" : bbox }, 
    "temporal" : {"intervals" : [{"snapshot" : date}]} 
}

In [34]:
from datetime import date, timedelta

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2016, 8, 1)
end_date = date(2016, 8, 14)

iso8601 = '%Y-%m-%dT%H:%M:%SZ'

for single_date in daterange(start_date, end_date):
    
    # Try Sentinel 2 first
    
    date = single_date.strftime(iso8601)
    querySentinelJson["temporal"]["intervals"][0]["snapshot"] = date
    print(querySentinelJson)
    sentinel_check = 0 # to check if there is Sentinel 2 data for this date or not
    
    try:    
        query = paw.PAIRSQuery(querySentinelJson, PAIRS_SERVER, PAIRS_CREDENTIALS, authType='api-key', overwriteExisting=False)
        query.submit()
        
        query.poll_till_finished()
        query.download()
        query.create_layers()
        # Could add some data manipulation here and then save the data
        sentinel_check = 1
    except:
        print('No Sentinel-2 data for this date')
        
    
    if sentinel_check:
        # Now download ERA 5 for dates of Sentinel 2
        date_start = single_date.strftime(iso8601)
        date_end = (single_date + timedelta(1)).strftime(iso8601)
        queryJson_era["temporal"]["intervals"][0]["start"] = date_start
        queryJson_era["temporal"]["intervals"][0]["end"] = date_end    
        #print(queryJson_era)

        try:    
            query = paw.PAIRSQuery(queryEraJson, PAIRS_SERVER, PAIRS_CREDENTIALS, authType='api-key', overwriteExisting=False)
            query.submit()

            query.poll_till_finished()
            query.download()
            query.create_layers()
            # Could add some data manipulation here and then save the data
        except:
            print('No ERA 5 data for this date')
    
    
        

INFO:ibmpairs.paw:PAIRS query JSON initialized.


{'layers': [{'type': 'raster', 'id': '49360'}, {'type': 'raster', 'id': '49681'}, {'type': 'raster', 'id': '49680'}, {'type': 'raster', 'id': '49361'}, {'type': 'raster', 'id': '49464'}, {'type': 'raster', 'id': '49362'}], 'spatial': {'type': 'square', 'coordinates': ['46.94599', '7.68597', '47.20599', '7.94597']}, 'temporal': {'intervals': [{'snapshot': '2016-08-01T00:00:00Z'}]}}


INFO:ibmpairs.paw:Query successfully submitted, reference ID: 1663560000_35135820
INFO:ibmpairs.paw:Here we go, PAIRS query result successfully downloaded as '1663560000_35135820_461aca71893f5626c0c1561ca61ea8d7.zip'.
INFO:ibmpairs.paw:Data acknowledgement successfully loaded, print with `self.print_data_acknowledgement()`
INFO:ibmpairs.paw:PAIRS meta data loaded from 'output.info'.
INFO:ibmpairs.paw:PAIRS query JSON initialized.


{'layers': [{'type': 'raster', 'id': '49360'}, {'type': 'raster', 'id': '49681'}, {'type': 'raster', 'id': '49680'}, {'type': 'raster', 'id': '49361'}, {'type': 'raster', 'id': '49464'}, {'type': 'raster', 'id': '49362'}], 'spatial': {'type': 'square', 'coordinates': ['46.94599', '7.68597', '47.20599', '7.94597']}, 'temporal': {'intervals': [{'snapshot': '2016-08-02T00:00:00Z'}]}}


INFO:ibmpairs.paw:Query successfully submitted, reference ID: 1663560000_35204245
INFO:ibmpairs.paw:Bummer, the PAIRS query finished, but you'll never be able to download anything, status code: '21' (NoDataFound)
INFO:ibmpairs.paw:PAIRS query JSON initialized.


...

INFO:ibmpairs.paw:Query successfully submitted, reference ID: 1663560000_35653976
INFO:ibmpairs.paw:Bummer, the PAIRS query finished, but you'll never be able to download anything, status code: '21' (NoDataFound)
INFO:ibmpairs.paw:PAIRS query JSON initialized.


No Sentinel-2 data for this date
{'layers': [{'type': 'raster', 'id': '49360'}, {'type': 'raster', 'id': '49681'}, {'type': 'raster', 'id': '49680'}, {'type': 'raster', 'id': '49361'}, {'type': 'raster', 'id': '49464'}, {'type': 'raster', 'id': '49362'}], 'spatial': {'type': 'square', 'coordinates': ['46.94599', '7.68597', '47.20599', '7.94597']}, 'temporal': {'intervals': [{'snapshot': '2016-08-12T00:00:00Z'}]}}


INFO:ibmpairs.paw:Query successfully submitted, reference ID: 1663560000_35687169
INFO:ibmpairs.paw:Bummer, the PAIRS query finished, but you'll never be able to download anything, status code: '21' (NoDataFound)
INFO:ibmpairs.paw:PAIRS query JSON initialized.


No Sentinel-2 data for this date
{'layers': [{'type': 'raster', 'id': '49360'}, {'type': 'raster', 'id': '49681'}, {'type': 'raster', 'id': '49680'}, {'type': 'raster', 'id': '49361'}, {'type': 'raster', 'id': '49464'}, {'type': 'raster', 'id': '49362'}], 'spatial': {'type': 'square', 'coordinates': ['46.94599', '7.68597', '47.20599', '7.94597']}, 'temporal': {'intervals': [{'snapshot': '2016-08-13T00:00:00Z'}]}}


INFO:ibmpairs.paw:Query successfully submitted, reference ID: 1663560000_35720178
INFO:ibmpairs.paw:Here we go, PAIRS query result successfully downloaded as '1663560000_35720178_904797b6a0ed797a65d3b719dfb75de3.zip'.
INFO:ibmpairs.paw:Data acknowledgement successfully loaded, print with `self.print_data_acknowledgement()`
INFO:ibmpairs.paw:PAIRS meta data loaded from 'output.info'.
